For this lab we will use the movie ratings file from our movielens dataset. This file contains user rating of different movies.

In [2]:
# load the dataset into an RDD to get started
input_rdd = sc.textFile("/FileStore/tables/movielens/ratings.csv")

In [3]:
# lets take a look and see what is in this data
input_rdd.take(10)

In [4]:
# like before we will filter out the header column and create an list-rdd
movie_ratings_rdd = input_rdd.filter(lambda line: 'userId' not in line)
movie_ratings_rdd_rdd = movie_ratings_rdd.map(lambda x: x.split(','))
movie_ratings_rdd_rdd.take(10)

### Transformations on RDD of key-value Pairs
Key value pair is as a data structure consisting of a two elements key and a value. Structuring your data in a key-value pair enables you to do further interesting transformation on them, as we will see in this section.

If we are interested to know which movie received the lowest rating overall we can solve with a simple key-value transformation. First we will create a key value pair of the movie id and the user rating

In [7]:
# create a (movie id, rating) key value pair
movie_ratings_rdd = movie_ratings_rdd_rdd.map(lambda x: (x[1],float(x[2])))

In [8]:
# Lets take a look at all the values in our dataset to see if we have any unexpected value we should remove
movie_ratings_rdd.map(lambda x: x[1]).distinct().collect()

as the data looks good we can proceed to finding out the count

### reduceByKey

The higher-order reduceByKey method takes an associative binary operator as input and reduces values with the same key to a single value using the specified binary operator.

A binary operator takes two values as input and returns a single value as output. An associative operator returns the same result regardless of the grouping of the operands.

The reduceByKey method can be used for aggregating values by key. For example, it can be used for calculating sum, product, minimum or maximum of all the values mapped to the same key.

In [11]:
# reduce by key (airline) to get the total departure delay per airline
ratings_sum_rdd = movie_ratings_rdd.reduceByKey(lambda value1, value2: value1 + value2)

In [12]:
# print out movie id's with low ratings
sorted(ratings_sum_rdd.collect(), key=lambda x: x[1])

### groupByKey

The groupByKey method returns an RDD of pairs, where the first element in a pair is a key from the source RDD and the second element is a collection of all the values that have the same key. It is similar to the groupBy method that we saw earlier. The difference is that groupBy is a higher-order method that takes as input a function that returns a key for each element in the source RDD. The groupByKey method operates on an RDD of key-value pairs, so a key generator function is not required as input.

** The groupByKey method should be avoided. It is an expensive operation since it may shuffle data. For most use cases, better alternatives are available. ** 

https://databricks.gitbooks.io/databricks-spark-knowledge-base/content/best_practices/prefer_reducebykey_over_groupbykey.html

### join
The join method takes an RDD of key-value pairs as input and performs an inner join on the source and input RDDs. It returns an RDD of pairs, where the first element in a pair is a key found in both source and input RDD and the second element is a tuple containing values mapped to that key in the source and input RDD.

In [15]:
# We want to get the movie names, not just the code
ratings_sum_rdd.first()

In [16]:
# lucky for us, we have a table that translates these ids into actual movie names
movies_names_rdd = sc.textFile("/FileStore/tables/movielens/movies.csv").filter(lambda x: 'movieId' not in x).map(lambda x: x.split(',')).map(lambda x: (x[0],x[1]))

In [17]:
movies_names_rdd.take(10)

In [18]:
# We can use join to translate the code into names
ratings_sum_rdd.join(movies_names_rdd).take(10)

In [19]:
# we can keep just the airlines name and total departure delay from the join
movie_ratings_sum = ratings_sum_rdd.join(movies_names_rdd).map(lambda line: (line[1][1], line[1][0])).collect()

In [20]:
# lets store it and print it sorted 
sorted(movie_ratings_sum, key=lambda x: x[1])

### Can we really say these are the worst movies? 
#### No: Because we should also consider how many people voted, not the total sum!

In [22]:
# This is your first excercise. Find the average movie rating per movie and print out a few of the lowest ones!
# HINT: Create another RDD containing the count for each movie ratings made and then join over with the movie ratings sum rdd